## TRABAJO PRACTICO 2: CHAT PARA CONSULTAR EL CV
ALUMNA: Lara Rosenberg

In [1]:
#Importamos las librerias necesarias
import os
import streamlit as st
from pinecone import Pinecone
from groq import Groq
from typing import List
import nltk
nltk.download("punkt_tab")
nltk.download("punkt")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\damia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\damia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Definimos las API KEY necesarias (Pinecone y Groq)
os.environ['PINECONE_API_KEY'] = ''
os.environ['GROQ_API_KEY'] = ''

In [ ]:
# Funcion para particionar el texto
def read_and_chunk_sentences(
    file_path: str,
    chunk_size: int = 40,
    overlap: int = 10
) -> List[str]:
    """
    Lee el archivo de texto, lo separa en oraciones y hace el chunk con overlap.

    Argumentos:
        file_path (str): Path al archivo de texto.
        chunk_size (int): Numero de oraciones por chunk.
        overlap (int): Numero de oraciones que estan en overlap.

    Returns:
        List[str]: Lista de chunks.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} does not exist.")

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    sentences = nltk.sent_tokenize(text, language='spanish')
    chunks = []
    i = 0
    while i < len(sentences):
        chunk = sentences[i:i+chunk_size]
        if chunk:
            chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

In [4]:
# Instanciamos las API KEY necesarias
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [5]:
# Inicializamos cliente Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Definimos el nombre del índice
index_name = "cvlrf-index"

# Verificamos si ya existe, si no lo creamos
if not pc.list_indexes() or index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model": "llama-text-embed-v2",
            "field_map": {"text": "chunk_text"}
        }
    )

# Nos conectamos a ese indice
index_pc = pc.Index(index_name)

# Leemos el archivo y lo dividimos en chunks
chunks = read_and_chunk_sentences("CV LARA ROSENBERG.txt", chunk_size=5, overlap=2)

# Armamos los documentos en el formato requerido
documents = []
for i, chunk in enumerate(chunks):
    documents.append({
        "_id": f"cv_chunk_{i + 1}",
        "chunk_text": chunk,
        "category": "cv"
    })


C:\Users\damia\Anaconda3\envs\nuevo_entorno\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Validamos que la metrica por default sea coseno
pc.describe_index(index_name)

{
    "name": "cvlrf-index",
    "metric": "cosine",
    "host": "cvlrf-index-b8qunng.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null,
    "embed": {
        "model": "llama-text-embed-v2",
        "field_map": {
            "text": "chunk_text"
        },
        "dimension": 1024,
        "metric": "cosine",
        "write_parameters": {
            "dimension": 1024.0,
            "input_type": "passage",
            "truncate": "END"
        },
        "read_parameters": {
            "dimension": 1024.0,
            "input_type": "query",
            "truncate": "END"
        },
        "vector_type": "dense"
    }
}

In [8]:
# Analizamos los documentos que se crearon
documents

[{'_id': 'cv_chunk_1',
  'chunk_text': 'DATOS PERSONALES\nLara Rosenberg \nBuenos Aires, Argentina \nEmail: lararosenberg21@gmail.com \nTeléfono: +5491133948355 \nFecha de Nacimiento: 7/10/1995\n\nRESUMEN PROFESIONAL:\nEspecialista en ciencia de datos y modelado de riesgo crediticio con más de 6 años de experiencia en la industria financiera y tecnológica. Trayectoria consolidada en el desarrollo e implementación de modelos predictivos, machine learning y análisis de grandes volúmenes de datos. Fuerte formación técnica, con enfoque en soluciones de negocio basadas en datos y métricas de performance. EXPERIENCIA PROFESIONAL:\n\nMERCADO LIBRE (Marzo 2023 - Actualidad)\n\nCredit Risk Modeling Supervisor (Marzo 2024 - Actualidad)  \n- Supervisión de estrategias de modelado de riesgo crediticio para productos financieros de la región. - Coordinación de equipo técnico y evaluación de performance de modelos.',
  'category': 'cv'},
 {'_id': 'cv_chunk_2',
  'chunk_text': 'EXPERIENCIA PROFESIONA

In [9]:
# Namespace donde se guardarán los vectores
namespace = "cvlrf-namespace"

# Subimos los chunks al índice --> lo dejamos comentado porque ya lo hicimos la primera vez que corrimos esto.
#index_pc.upsert_records(namespace=namespace, records=documents)
#print(f"Subidos {len(documents)} chunks al índice.")

In [10]:
def search_similar(text, top_k=10, namespace="cvlrf-namespace", debug = False):
    """
    Función para buscar los chunks de texto más similares a una consulta utilizando el indice.

    Argumentos:
    - text (str): La consulta o pregunta del usuario.
    - top_k (int, opcional): Número de resultados más relevantes que se deben devolver. Por defecto es 10.
    - namespace (str, opcional): El espacio de nombres donde se guardan los vectores. Por defecto es "cvlrf-namespace".
    - debug (bool, opcional): Si se debe imprimir información de depuración sobre los resultados. Por defecto es False.

    Returns:
    - List[str]: Una lista de los chunks de texto más relevantes (en formato string) basados en la consulta.
    """
    # Search the dense index
    results = index_pc.search(
        namespace=namespace,
        query={
            "top_k": top_k,
            "inputs": {
                'text': text
            }
        }
    )

    # Print the results
    data = []
    for hit in results['result']['hits']:
        tmp = f"id: {hit['_id']:<5} | score: {round(hit['_score'], 2):<5} | category: {hit['fields']['category']:<10} | text: {hit['fields']['chunk_text']:<50}"
        if debug:
            print(tmp)
        data.append(tmp)

    return data


In [11]:
client = Groq(api_key=GROQ_API_KEY)

class ChatSession:
    def __init__(self, client, model="meta-llama/llama-4-scout-17b-16e-instruct"):
        self.client = client
        self.model = model
        self.messages = []

    def add_user_message(self, content):
        self.messages.append({"role": "user", "content": content})

    def add_assistant_message(self, content):
        self.messages.append({"role": "assistant", "content": content})

    def chat(self, msg, temperature=1, max_completion_tokens=1024, top_p=1, stream=True, stop=None):
        context_text = ""
        context = search_similar(msg, top_k=10, debug=False)
        context_text = "\n\n".join(context)
        prompt = f"""
Sos un asistente conversacional. Respondé de forma natural y conversacional.

Sin embargo, si la pregunta se refiere a Lara Rosenberg o su currículum vitae, usá exclusivamente la información del siguiente contexto para responder. Si no encontrás la respuesta en el contexto, decilo claramente. Si la pregunta no tiene que ver con Lara, respondé normalmente sin usar el contexto.
Consulta previa (historial): {self.messages}

Consulta: {msg}

Contexto:
{context_text}
"""
        self.add_user_message(prompt.strip())

        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=temperature,
            max_completion_tokens=max_completion_tokens,
            top_p=top_p,
            stream=stream,
            stop=stop
        )

        response = ""
        for chunk in completion:
            delta = chunk.choices[0].delta.content or ""
            response += delta
        self.add_assistant_message(response)
        return response

Testeamos el funcionamiento

In [12]:
# 1. Hacemos una consulta
query = "¿Si estamos en 2025, cuantos anos tiene Lara?"

# 2. Creamos sesión de chat
chat = ChatSession(client)

# 3. Obtenemos respuesta generada
respuesta = chat.chat(query)
print(respuesta)


Si estamos en 2025 y Lara Rosenberg nació el 7 de octubre de 1995, eso significa que en 2025 tendría 30 años.


In [13]:
query = "¿Que estudiaba Lara en 2015?"
respuesta = chat.chat(query)
print(respuesta)

Según el contexto, Lara estudió Actuario en Economía en la Universidad de Buenos Aires desde 2014 hasta 2019. Por lo tanto, en 2015, Lara estaba estudiando Actuario en Economía.


In [14]:
query = "¿Cual fue el ultimo trabajo de Lara?"
respuesta = chat.chat(query)
print(respuesta)

Según el contexto, el último trabajo mencionado de Lara Rosenberg es como Credit Risk Modeling Supervisor en Mercado Libre desde marzo 2024 hasta la actualidad.


In [15]:
query = "¿En cuantas empresas trabajo Lara?"
respuesta = chat.chat(query)
print(respuesta)

Según el contexto, Lara Rosenberg ha trabajado en varias empresas:

1. Mercado Libre (Marzo2023 - Actualidad)
2. ALGORITHIA (Noviembre2021 - Marzo2023)
3. CÍRCULO DE CRÉDITO (Marzo2020 - Noviembre2021)
4. BANCO PATAGONIA (Septiembre2019 - Marzo2020)
5. INSTITUTO ARGENTINO DE MERCADO DE CAPITALES (Agosto2018 - Agosto2019)
6. METLIFE SEGUROS (Abril2018 - Agosto2018)

Por lo tanto, Lara ha trabajado en 6 empresas diferentes.
